In [2]:
from pettingzoo.mpe import simple_spread_v3

env = simple_spread_v3.env(render_mode="human")
env.reset(seed=42)

for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()

    if termination or truncation:
        action = None
    else:
        # this is where you would insert your policy
        action = env.action_space(agent).sample()

    env.step(action)
env.close()

In [8]:
env = simple_spread_v3.env(render_mode="human")
env.reset(seed=42)
print(env.agents)
print(env.agent_selection)

['agent_0', 'agent_1', 'agent_2']
agent_0
